In [3]:
from datasets import load_dataset
import json
import requests
from collections import defaultdict

from langchain.llms import OpenAI
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from inference_api import YiyanInferenceApi, HfInferenceApi
import regex

dataset_mapping_url = f"https://raw.githubusercontent.com/SJTU-LIT/ceval/main/subject_mapping.json"

llm = YiyanInferenceApi("yiyan-007", debug=True)
# llm = OpenAI(model_name="text-davinci-003")

def get_data_mapping(mapping_url):
    headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    response = requests.request("GET", mapping_url, headers=headers)
    return json.loads(response.content.decode('utf-8'))

def escape_format(text : str) -> str :
    return text.replace('{', '{{').replace('}', '}}')

def gen_by_fewshot_prompt(task_name, fewshot_set, test_set) :

    prompt_template = """问题：{question} \n选项\nA: {A}\nB: {B}\nC: {C}\nD: {D}\n思考：{explanation}\n答案：{answer}\n\n"""

    example_prompt = PromptTemplate(
        input_variables = ['question', 'A', 'B', 'C', 'D', 'explanation', 'answer'],
        template = prompt_template
    )
    
    fewshot_examples = []
    for id in range(len(fewshot_set)) :
        fewshot_examples.append({'question': escape_format(fewshot_set[id]['question']), 
                                 'A': escape_format(fewshot_set[id]['A']),
                                 'B': escape_format(fewshot_set[id]['B']),
                                 'C': escape_format(fewshot_set[id]['C']),
                                 'D': escape_format(fewshot_set[id]['D']),
                                 'explanation': escape_format(fewshot_set[id]['explanation']), 
                                 'answer': fewshot_set[id]['answer']})

    example_selector = LengthBasedExampleSelector(
        examples = fewshot_examples,
        example_prompt = example_prompt, 
        max_length = 1024,
        get_text_length = lambda x: len(x) # 2 tokens per Chinese character
    )

    few_shot_prompt = FewShotPromptTemplate(
        example_selector = example_selector,
        example_prompt = example_prompt,
        prefix = f"请作为一个{task_name}科目的考生，参考示例回答单项选择题，示例如下：\n", 
        suffix = """下面是要回答的问题，可以先写出思考过程，然后给出ABCD的一个选项作为答案，确保用<答案：>作为答案的前缀\n
                    问题：{question}\n选项\nA: {A}\nB: {B}\nC: {C}\nD: {D}\n思考：""",

        input_variables = ['question', 'A', 'B', 'C', 'D'], 
        example_separator = "\n\n"
    )

    correct, total = 0, 0
    for id in range(len(test_set)) :
        try :
            print(f"question: {test_set[id]['question']}")
            grounded_prompt = few_shot_prompt.format(question= escape_format(test_set[id]['question']),
                                                            A = escape_format(test_set[id]['A']),
                                                            B = escape_format(test_set[id]['B']),
                                                            C = escape_format(test_set[id]['C']),
                                                            D = escape_format(test_set[id]['D']))
            completion = llm(grounded_prompt)
            llm_answer = extract_answer(completion)
            answer = test_set[id]['answer']
            print(f"question: {id}, llm_answer={llm_answer}, answer={answer}")
            total += 1
            if llm_answer == answer :
                correct += 1
        except Exception as e :
            print(f"Exception in answer generation for question <{id}> in task <{task_name}>")

    return (correct, total)

def extract_answer(completion) :
    
    answer_prefixes = ['答案：', '答案为', '答案是', '故选', '选项中最接近的是', '选项为', '答案应该是']
    for answer_prefix in answer_prefixes :
        if answer_prefix in completion :
            pos = completion.find(answer_prefix) + len(answer_prefix)
            if pos < len(completion) :
                answer = completion[pos]
                if answer == '{' and pos + 1 < len(completion) :
                    answer = completion[pos + 1]
                if answer in ['A', 'B', 'C', 'D'] :
                    return answer
    return completion

data_mapping = get_data_mapping(dataset_mapping_url)
g_correct, g_total = 0.0, 0.0

tested_subject = 0
topK = len(data_mapping.keys())

for name in data_mapping.keys() :
    dataset=load_dataset(r"ceval/ceval-exam",name=name)
    (correct, total) = gen_by_fewshot_prompt(data_mapping[name][1], dataset['dev'], dataset['val'])
    g_correct += correct
    g_total += total
    print(f"Task <{name}>: correct={correct}, total={total}")
    print(f"Global accuracy so far: {g_correct/g_total:.4f}(g_correct={g_correct}, g_total={g_total})")

    tested_subject += 1
    if tested_subject >= topK :
        break

Yiyan api key: 6Rs1EueWFULgAnPUS2HQDx2H, Yiyan sec key: Fl3R1XOeBdQfIcHng4Ah25RHzuSQxChT


Found cached dataset ceval-exam (/Users/wujianmin/.cache/huggingface/datasets/ceval___ceval-exam/college_physics/1.0.0/955d76b4f8ea8c9b6a14519905c8c10f781d3feb698126e274d2eb16cefc671f)


  0%|          | 0/3 [00:00<?, ?it/s]

{'id': 1, 'question': '有三个直径相同的金属小球，小球1和2带等量同号电荷，两者的距离远大于小球直径，相互作用力为F。小球3不带电，装有绝缘手柄．用小球3先和小球1碰一下，接着又和小球2碰一下，然后移去。则此时小球1和2之间的相互作用力为____', 'A': 'F/4', 'B': '3F/8', 'C': 'F/2', 'D': '3F/4', 'answer': 'B', 'explanation': ''}
fewshot_len: 0
question: α粒子在加速器中被加速，当其质量为静止质量的 3 倍时，其动能为静止能量的____
template: 请作为一个大学物理科目的考生，参考示例回答单项选择题，示例如下：


问题：有一劲度系数为k的轻弹簧，竖直放置，下端悬一质量为m的小球，开始时使弹簧为原长而小球恰好与地接触，今将弹簧上端缓慢地提起，直到小球刚能脱离地面为止，在此过程中外力作功为____ 
选项
A: m^2g^2/(4k)
B: m^2g^2/(3k)
C: m^2g^2/(2k)
D: 2m^2g^2/k
思考：1. 当小球刚好脱离地面时，弹簧的弹力与小球所受重力相等，即$kx = mg$，其中x表示弹簧的伸长量。
2. 由上述方程可得，$x = mg/k$。
3. 外力作功等于弹簧势能的增加，即$W = \frac{{1}}{{2}}kx^2 - 0$（因为开始时弹簧为原长，势能为0）。
4. 将第2步中得到的x代入第3步的公式，得到$W = \frac{{1}}{{2}}k(mg/k)^2 = m^2g^2/(2k)$。
答案：C



问题：某质点作直线运动的运动学方程为 x=3t-5t^3 +6(SI)，则该质点作____ 
选项
A: 匀加速直线运动，加速度沿x轴正方向
B: 匀加速直线运动，加速度沿x轴负方向
C: 变加速直线运动，加速度沿x轴正方向
D: 变加速直线运动，加速度沿x轴负方向
思考：1. 首先，我们需要求出速度和加速度。速度是位置关于时间的导数，加速度是速度关于时间的导数。对运动学方程求导，得到速度方程v = dx/dt = 3 - 15t^2，再求导得到加速度方程a = dv/dt = -30t。
2. 接着，我们分析加速度方程a = -30t，这说明加速度随时间变化，且沿x轴负方向

KeyboardInterrupt: 